In [1]:
from datasets import load_from_disk
train = load_from_disk("train_220K.hf")
test = load_from_disk("test_220K.hf")

In [2]:
train = train.shuffle(seed=42)
test = test.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\train_220K.hf\cache-e9109fd02d9e5f0c.arrow
Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\test_220K.hf\cache-41f2bd74d9d4d4e3.arrow


In [3]:
from transformers import AutoTokenizer
from pathlib import Path
roberta_model_name = "HooshvareLab/roberta-fa-zwnj-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name,cache_dir=Path.cwd())

In [4]:
from transformers import RobertaConfig, RobertaModel
from pathlib import Path
roberta_model = RobertaModel.from_pretrained(roberta_model_name)
roberta_config = RobertaConfig.from_pretrained(roberta_model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at HooshvareLab/roberta-fa-zwnj-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from transformers.models.roberta.modeling_roberta import RobertaModel
from torch import nn
class ParsRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = roberta_config
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.parsbert = RobertaModel(config)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()
    
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        
        # Use model body to get encoder representations
        outputs = self.parsbert(input_ids, attention_mask=attention_mask,
        token_type_ids=token_type_ids, **kwargs)
        
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions)
        

In [6]:
ner_tags = train["ner"]
ner_tag_names = set(tag for tags in ner_tags for tag in tags)
                
index2tag = {idx: tag for idx, tag in enumerate(ner_tag_names)}
tag2index = {tag: idx for idx, tag in enumerate(ner_tag_names)}

In [7]:
from transformers import AutoConfig

roberta_config = AutoConfig.from_pretrained(roberta_model_name,num_labels=4,
                                            id2label=index2tag, label2id=tag2index)

In [8]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
per_text = " ".join(train['words'][0])
input_ids = roberta_tokenizer.encode(per_text, return_tensors="pt")
roberta_tokens = roberta_tokenizer(per_text).tokens()
pd.DataFrame([roberta_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55
Tokens,<s>,He,Ġl,ater,Ġreturn,ed,Ġto,Ġthe,ĠUnited,ĠSt,ates,Ġwhere,Ġhe,Ġwas,Ġcal,led,Ġin,Ġby,ĠD,.,C,.,ĠUnited,"Ġ,",Ġand,Ġplay,ed,Ġse,ver,al,Ġres,erve,Ġg,ames,Ġag,ain,st,ĠCh,ic,ago,ĠFire,"Ġ,",ĠNew,ĠEng,land,ĠRev,olution,Ġand,ĠCol,umb,us,ĠCre,w,"Ġ,",Ġ.,</s>
Input IDs,0,22606,4486,12480,32617,2156,4738,3497,36300,3700,16267,37988,11742,18536,31629,17145,4142,13474,1410,24,45,24,36300,4221,4517,28918,2156,20796,4823,1353,14426,31058,4998,11323,23277,6005,1731,5475,1699,34133,15592,4221,10445,14437,12630,30916,20077,4517,13055,22420,2831,11040,97,4221,19225,2


In [9]:
from transformers import RobertaTokenizerFast
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(roberta_model_name, add_prefix_space=True)

def tokenize_and_align_labels(examples):
    tokenized_inputs = roberta_tokenizer(examples["words"], truncation=True,
    is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_token = label[word_idx]
                # Use the label map to get the numerical value for each entity
                label_ids.append(tag2index[label_token])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels,batched=True,
                      remove_columns=['id','words','lang','ner','ratio'])

In [11]:
train_encoded = encode_panx_dataset(train)
test_encoded = encode_panx_dataset(test)

Map:   0%|          | 0/171745 [00:00<?, ? examples/s]

Map:   0%|          | 0/42937 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoConfig

roberta_config = AutoConfig.from_pretrained(roberta_model_name,num_labels=4,
                                            id2label=index2tag, label2id=tag2index)

In [13]:
train_encoded

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 171745
})

In [14]:
import numpy as np

def align_predictions(predictions, label_ids):
        preds = np.argmax(predictions, axis=2)
        batch_size, seq_len = preds.shape
        labels_list, preds_list = [], []
        for batch_idx in range(batch_size):
            example_labels, example_preds = [], []
            for seq_idx in range(seq_len):
                # Ignore label IDs = -100
                if label_ids[batch_idx, seq_idx] != -100:
                    example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                    example_preds.append(index2tag[preds[batch_idx][seq_idx]])
            labels_list.append(example_labels)
            preds_list.append(example_preds)
        return preds_list, labels_list

In [15]:
from transformers import TrainingArguments
import torch

num_epochs = 2
batch_size = 18
logging_steps = len(train_encoded) 

training_args = TrainingArguments(
    output_dir="output", log_level="error", num_train_epochs=num_epochs,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
    eval_accumulation_steps=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=18,
    seed=42,
    logging_strategy="steps", evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False)


In [16]:
def model_init():
    return (ParsRobertaForTokenClassification
                  .from_pretrained(roberta_model_name, config=roberta_config,cache_dir=Path.cwd())
                  .to(device))

In [17]:
from transformers import DataCollatorForTokenClassification

class CustomDataCollator(DataCollatorForTokenClassification):
    def __call__(self, features):
        batch = super().__call__(features)
        # Filter out the '-100' label IDs (padding tokens)
        batch["labels"] = torch.where(batch["labels"] != -100, batch["labels"], -100)
        return batch

# Use the custom data collator
data_collator = DataCollatorForTokenClassification(tokenizer=roberta_tokenizer)

In [18]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
    eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [19]:
from transformers import Trainer
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = Trainer(model_init=model_init, args=training_args,
                    data_collator=data_collator, compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=test_encoded,
                    tokenizer=roberta_tokenizer)


In [20]:
result = trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.177999,0.592254
2,No log,0.152724,0.660872


C:\Users\11\anaconda3\envs\Pytorch\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\11\anaconda3\envs\Pytorch\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\11\anaconda3\envs\Pytorch\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [21]:
import torch
torch.cuda.empty_cache()